In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary

import numpy as np
import pickle
import tqdm

import itertools
from itertools import islice


In [2]:
import urllib.request
import shutil
import zipfile
import os
import pickle
from collections import Counter
import numpy as np

In [3]:
# %load_ext autoreload
# %autoreload 2
from column_models import *

In [4]:

url = 'http://mattmahoney.net/dc/text8.zip'
filename = 'text8.zip'
train_size = 99000000

if not os.path.isfile(filename):
    print('Downloading text8 dataset...')

    with urllib.request.urlopen(url) as response, \
        open(filename, 'wb') as outfile:
        shutil.copyfileobj(response, outfile)

rawdata = zipfile.ZipFile(filename).read('text8').decode('utf-8')

In [5]:
len(rawdata)

100000000

In [6]:
rawdata[:200]

' anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term '

In [7]:
# train_split = rawdata[:train_size]
# valid_split = rawdata[train_size:]

# vocab = Counter()

In [8]:

# for c in train_split:
#     vocab[c] += 1

# vocab_cut = {k: v for k, v in vocab.items() if v > 10}
# vocab_sorted = sorted(vocab_cut.items(), key=lambda x: x[1], reverse=True)
# wordmap = {k: id + 1 for id, (k, _) in enumerate(vocab_sorted)}


In [9]:
# len(wordmap)

In [10]:
# wordmap

In [11]:
# word_to_ix = {"hello": 0, "world": 1}
# embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
# lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
# hello_embed = embeds(lookup_tensor)
# print(hello_embed)

In [12]:
# print(embeds(torch.tensor([0,0,0,1], dtype=torch.long)))
# print(embeds(torch.tensor(1, dtype=torch.long)))

In [13]:
## This is not THE most efficient thing, is a step to learn ... alter will deal with memory issues
# to change things for the better the input text should be read by chunks

# splicing through the text such as to create the input vectors
def get_next_batch_txt(in_txt, index=0, char_len=1024, step=128, batch_size=128):
    """
    Returns a generator of batches for the given input.
    
    :param in_txt: text used to generate the tensor data
    :param index: index where to begin the next batch
    :param char_len: number of characters lenght of the tensor
    :param step: number of characters to jump for the next step (we don't want to repeat too many things ... )
    :param batch_size: number of samples in each batch
    
    :returns A generator of batches for the given input with dimension [seq_len,batch_size,1]
    
    """
    ret = []
    for b in range(batch_size):
        # get the new part
#         print("get next ", index)
        ret.append(in_txt[index:min(index+char_len, len(in_txt)-1)])
        # go to next
        index += step
        if index >= len(in_txt) - step:
            index = -1
            break
        
    # tensorize
    
    # TODO now make sure all the parts are equal, else complete with
    return ret, index
    
# simple tokenizer, should do something to make it fast instead of char by char ... ?
def tokenize_txt(txt, tokendict, dim):
    """
    
    """
    ret = np.empty(dim)
    for i in range(len(txt)):
        c = txt[i]
        if c in tokendict:
            ret[i] = tokendict[c]
        else:
            ret[i] = 0
    return ret
    

def tokenize_batch(batch, tokendict, dim):
    """
    """
    ret = []
    for b in batch:
        ret.append(tokenize_txt(b, tokendict,dim))
    return np.stack(ret)    
    
    
def tensorize_batch(batch):
    """
    
    """
    # shape should be ?? (batch_size, one_hot, sequence_width) <<- the first one and NOT: (seq_len, batch_size, input_size)
    # so as an example could be: (1024, 128, 1) where there are 1024 for each of the 128 batches and each char is dim 1
    # this is BEFORE the embedding layer
    #TODO
#     ret = torch.tensor(batch.transpose(0,1)).unsqueeze(1).long()
#     ret = torch.tensor(batch.transpose(0,1)).unsqueeze(0).long()
#     ret = torch.tensor(batch.transpose(0,1)).long()
    ret = torch.tensor(batch).long()  # .transpose(1,2)
    return ret
    
    

In [14]:
def get_all_batches(txt, token_dict, char_len=1024, step=128, batch_size=128):
    index = 0
#     batches = []
    while index >= 0:
#         print(index)
        batch, index = get_next_batch_txt(txt, index, char_len, step, batch_size)
        tok = tokenize_batch(batch, token_dict, char_len)
        tens = tensorize_batch(tok)
#         batches.append(batch)  # yield batch
        yield batch, tok, tens
#     return batches

In [15]:
# list(get_all_batches(rawdata[:100], txt2num_2seg, 20, 5, 12))

In [16]:
# batch_gen = get_all_batches(rawdata[:100], txt2num_2seg, 1024, 128, 128)

In [17]:
in_txt = rawdata
index=0
char_len=1024
step=128
batch_size=128

In [18]:
# txtbatch = get_next_batch_txt(in_txt, index=0, char_len=1024, step=128, batch_size=128)
# tok_batch = tokenize_batch(txtbatch, txt2num_2seg, char_len)
# ten_batch = tensorize_batch(tok_batch)

In [19]:
# utf8codes.shape

In [20]:
# embeds = nn.Embedding(*(utf8codes.shape))

In [21]:
# embeds.weight.data.copy_(torch.from_numpy(utf8codes))

In [22]:
# print(embeds(torch.tensor([0,0,0,1], dtype=torch.long)))


In [23]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [24]:
ls ../utf8-codes/

code2txt_2seg.pkl.pkl  txt2code_2seg.pkl.pkl  utf8_code_matrix_2seg.pkl.npy
code2txt_3seg.pkl.pkl  txt2code_3seg.pkl.pkl  utf8_code_matrix_3seg.pkl.npy
num2txt_2seg.pkl.pkl   txt2num_2seg.pkl.pkl
num2txt_3seg.pkl.pkl   txt2num_3seg.pkl.pkl


In [25]:
# load the UTF-8 codes 
utf8codes = np.load("../utf8-codes/utf8_code_matrix_2seg.pkl.npy")
txt2code_2seg = load_obj("../utf8-codes/txt2code_2seg.pkl")
code2txt_2seg = load_obj("../utf8-codes/code2txt_2seg.pkl")
txt2num_2seg = load_obj("../utf8-codes/txt2num_2seg.pkl")
num2txt_2seg = load_obj("../utf8-codes/num2txt_2seg.pkl")


In [26]:
txt2num_2seg['a']

97

In [27]:
utf8codes.shape

(1984, 324)

In [28]:
# model = FFColNet(utf8codes)

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [30]:
device

device(type='cuda', index=0)

In [31]:
def train(model, criterion, optimizer, batch_gen):

    for b,tok,x in batch_gen:
#         x = x.unsqueeze(2).to(device)
        x = x.long().to(device)
        y = x
#         print(x.shape, y.shape)
        model.zero_grad()
        output = model(x)
        ycode = model.embeds(y.long()[:,-128:])
#         print(output.shape, ycode.shape, y.shape)
        loss = criterion(output, ycode)
        loss.backward()
#         clip_global_norm(model, 0.25)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)

        optimizer.step()
        print('Loss: {:.5f}'.format(loss.item()))
        break

In [32]:
# def prime_factors(n):
#     i = 2
#     factors = []
#     while i * i <= n:
#         if n % i:
#             i += 1
#         else:
#             n //= i
#             factors.append(i)
#     if n > 1:
#         factors.append(n)
#     return factors


In [33]:
# prime_factors(324)

In [34]:
len(rawdata)

100000000

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [36]:
# import importlib
# importlib.reload('column_models.MixedConvAttentionColumn')
# model = FFColNet(utf8codes,1024).to(device)
# model = NLPGatedConv1DColumnV1(utf8codes, 324).to(device)
# model = MixedConvAttentionColumn(utf8codes, c_in=[324, 64, 128, 128])  # .to(device)
# model = MixedConvLinearColumns(utf8codes)
# model = MixedConvAttentiveColumns(utf8codes,
#                                   channels=[64, 64, 128, 128, 64],  # channels for blocks
#                                  b_layers=[5, 5, 5],  # number of layers for each bloc
#                                  c_attentive=[64, 64, 64, 64, 64],  # in and out channels of the linear layers inputs
#                                  )
model = MixedColsTest()
# model = MixedColsTest(utf8codes)
# model = ConvColumn(utf8codes, c_in=[324, 64, 128, 128]).to(device)
model = model.to(device)

In [37]:
count_parameters(model)

2643456

In [38]:
criterion = nn.BCEWithLogitsLoss().to(device)  #The problem was the loss function... CrossEntropyLoss()
# criterion = nn.NLLLoss().to(device)
# optimizer = optim.Adagrad(model.parameters(), lr=0.001, lr_decay=1e-5, weight_decay=1e-5)
optimizer = optim.Adam(model.parameters(),lr=1e-3)

In [39]:
batch_gen = get_all_batches(rawdata, txt2num_2seg, 1024, 128, 40)

In [40]:
%%time
%time train(model, criterion, optimizer, batch_gen)

Embedding:  torch.Size([40, 1024])
Encoding:  torch.Size([40, 32, 1024])
1 mixcol torch.Size([40, 32, 1024]) torch.float32
4 mixcol torch.Size([40, 64, 1024])
7 mixcol torch.Size([40, 32, 128])
decanlp:  torch.Size([40, 32, 16]) torch.Size([40, 32, 16]) torch.Size([40, 32, 16])
decanlp:  torch.Size([40, 32, 16]) torch.Size([40, 32, 16]) torch.Size([40, 32, 16])
decanlp:  torch.Size([40, 32, 16]) torch.Size([40, 32, 16]) torch.Size([40, 32, 16])
decanlp:  torch.Size([40, 32, 16]) torch.Size([40, 32, 16]) torch.Size([40, 32, 16])
decanlp:  torch.Size([40, 32, 16]) torch.Size([40, 32, 16]) torch.Size([40, 32, 16])
decanlp:  torch.Size([40, 32, 16]) torch.Size([40, 32, 16]) torch.Size([40, 32, 16])
decanlp:  torch.Size([40, 32, 16]) torch.Size([40, 32, 16]) torch.Size([40, 32, 16])
decanlp:  torch.Size([40, 32, 16]) torch.Size([40, 32, 16]) torch.Size([40, 32, 16])
9 mixcol torch.Size([40, 32, 128])
decanlp:  torch.Size([40, 64, 16]) torch.Size([40, 64, 16]) torch.Size([40, 64, 16])
decanl

RuntimeError: size mismatch, m1: [5120 x 32], m2: [128 x 128] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:273

CPU times: user 114 ms, sys: 48.6 ms, total: 163 ms
Wall time: 122 ms


In [41]:
# b0 = list(batch_gen)[0]

In [42]:
# summary(model, (1, 256,1024))

In [43]:
131072/1024.

128.0